<a href="https://colab.research.google.com/github/hijera/NN_colab_hw/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_%C2%AB%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%B2_%D0%90%D0%9E%D0%A2%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Сделать классификацию данных fakenews
1. Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

In [ ]:
import pandas as pd
df = pd.read_csv('Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
#Преобразуем label в 2 класса, 0 если fake, 1 если real
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Encode labels in column 'species'.
df['label_c']= label_encoder.fit_transform(df['label'])
df.head()

,id,tweet,label,label_c
0,1,The CDC currently reports 99031 deaths. In gen...,real,1
1,2,States reported 1121 deaths a small rise from ...,real,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,1
4,5,Populous states can generate large case counts...,real,1


In [ ]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Антон\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|████████████████████████████████████████████████████████████████████████████| 6420/6420 [00:00<00:00, 6744.36it/s]


In [ ]:
from gensim.models.word2vec import Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4,  min_count=3, window=5,vector_size=300,epochs=20)

CPU times: total: 4.52 s
Wall time: 1.42 s


In [ ]:
import numpy as np
model_tweets.init_sims()
def get_text_embedding(text,fsize=300):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(fsize)
    return result

C:\Users\Антон\AppData\Local\Temp\ipykernel_17652\2538727259.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims()


In [ ]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|████████████████████████████████████████████████████████████████████████████| 6420/6420 [00:01<00:00, 4862.54it/s]


In [ ]:
features[0]

array([ 4.13011789e+00,  1.02443466e+01, -5.47356987e+00, -9.64809418e+00,
       -1.44260550e+00, -5.19524956e+00, -1.28986359e+00,  1.22126780e+01,
       -1.21712422e+00, -1.77339983e+00,  7.14585257e+00, -1.93240309e+00,
       -3.40784740e+00, -1.03475559e+00,  3.25792503e+00,  1.28895450e+00,
        1.27556229e+01,  1.51693165e+00, -3.43241304e-01,  7.22369909e+00,
       -7.79073048e+00,  4.46420765e+00,  1.10555897e+01,  8.70104790e+00,
        5.62481451e+00,  1.02765446e+01, -5.02773428e+00,  6.04070807e+00,
        2.94398141e+00, -4.71269894e+00,  9.28076935e+00, -9.04053450e-01,
       -8.25742531e+00, -9.09383416e-01, -1.56467155e-01, -3.40947032e-01,
       -2.85702497e-01, -5.70674610e+00,  6.38934851e+00, -3.92306709e+00,
       -6.47618484e+00, -3.44814992e+00,  3.74931669e+00, -6.22497678e-01,
       -6.53364563e+00,  4.91021061e+00,  2.38904619e+00,  9.45340037e-01,
       -9.44050789e+00,  7.70023632e+00, -3.73809934e+00,  2.19432139e+00,
       -6.44470358e+00, -

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn import tree
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label_c, test_size=0.33)

In [ ]:
#Используем SVC
model = SVC()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1034
           1       0.92      0.94      0.93      1085

    accuracy                           0.92      2119
   macro avg       0.93      0.92      0.92      2119
weighted avg       0.93      0.92      0.92      2119



In [ ]:
#Используем AdaBoost
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=256, algorithm="SAMME",)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1034
           1       0.92      0.94      0.93      1085

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [ ]:
#KNeghbours
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=11)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1034
           1       0.92      0.93      0.92      1085

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [ ]:
netsize=150
embsize=400

In [ ]:
#Попробуем собрать нейросеть на pytorch
def get_word_embedding(tokens, max_len=200,tweets=model_tweets,fsize=300):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in tweets.wv:
                result.append(tweets.wv[word])
            else:
                result.append(np.zeros(fsize))
        else:
            result.append(np.zeros(fsize))
    return result


In [ ]:
model_tweets_pt = Word2Vec(sentences, workers=4,  min_count=3, window=5,vector_size=netsize,epochs=20)
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
features = [get_word_embedding(text, embsize,model_tweets_pt,netsize) for text in tqdm(token_lists)]
X_train, X_test, y_train, y_test = train_test_split(features, df.label_c, test_size=0.33)

100%|████████████████████████████████████████████████████████████████████████████| 6420/6420 [00:01<00:00, 3601.51it/s]


In [ ]:
print(np.array(X_train).shape)
print(np.array(y_train).shape)

(4301, 400, 150)
(4301,)


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter

class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()
        self.lstm= nn.LSTM(netsize,150)
        self.linear1 = nn.Linear(netsize,75) #
        self.relu1 = nn.ReLU() #
        self.linear2 = nn.Linear(75,50)#
        self.relu2 = nn.ReLU() #
        self.linear3 = nn.Linear(50,16)
        self.relu3 = nn.ReLU() #

        self.linear4 = nn.Linear(16,1) # IN 20 OUT 2 | OUTPUT


    def forward(self,x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        out = self.linear1(longterm) # Input Layer
 #       out= self.linear1(x)
        #out = self.relu1(out)

        out = self.linear2(out) # Hidden Layer
        #out = self.relu2(out)
        out = self.linear3(out) # Hidden Layer
        #out = self.relu3(out)
        out = self.linear4(out) # Hidden Layer

        prediction = torch.sigmoid(out)
        return prediction


net = Net()
print(net)

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(np.array(y_train)).float() #.type(torch.LongTensor)
optimizer = optim.AdamW(net.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
in_data.shape

In [ ]:
targets.shape

In [ ]:
#X_train = torch.Tensor(X_train)

# You must convert it into LongTensor. I did it once
#y_train = torch.Tensor(y_train).type(torch.LongTensor)

#X_test = torch.Tensor(X_test)
#y_test = torch.Tensor(y_test)
#error = nn.CrossEntropyLoss()
#EPOCHS = 20

#for epoch in range(EPOCHS):

    # Clearing gradients
#    optimizer.zero_grad()

    # Forward Propagation
#    outs = net(X_train)

    # Computing Loss
#    loss = criterion(outs,y_train.unsqueeze(-1))

    # Backward propagation
#    loss.backward()

    # Updating parameters
#    optimizer.step()

    # Printing loss
#    print(f"Loss after iteration {epoch} is {loss}")

In [ ]:
def train_one_epoch(in_data, targets, batch_size=8):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        #print(batch_x.shape)
        #print(batch_y.shape)
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)


In [ ]:
for i in range(5):
    train_one_epoch(in_data, targets,8)

100%|████████████████████████████████████████████████████████████████████████████████| 538/538 [05:41<00:00,  1.58it/s]


tensor(0.6617, grad_fn=<BinaryCrossEntropyBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 538/538 [05:40<00:00,  1.58it/s]


tensor(0.6625, grad_fn=<BinaryCrossEntropyBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 538/538 [05:40<00:00,  1.58it/s]


tensor(0.6627, grad_fn=<BinaryCrossEntropyBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 538/538 [05:41<00:00,  1.58it/s]


tensor(0.6627, grad_fn=<BinaryCrossEntropyBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 538/538 [05:40<00:00,  1.58it/s]

tensor(0.6628, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(np.array(y_test)).float()
with torch.no_grad():
    output = net(in_data_test).reshape(-1)
result = (output > 0.5) == targets_test
result.sum().item() / len(result)

0.5176970268994809

In [ ]:
#Еще чуть чтуь покрутим?
for i in range(2):
    train_one_epoch(in_data, targets,64)

100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [05:12<00:00,  4.59s/it]


tensor(0.7061, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [05:13<00:00,  4.61s/it]

tensor(0.7061, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(np.array(y_test)).float()
with torch.no_grad():
    output = net(in_data_test).reshape(-1)
result = (output > 0.5) == targets_test
result.sum().item() / len(result)

0.5176970268994809